In [1]:
import re
 
MAIL_REGEX = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
 
def checkMail(email):
    if(re.fullmatch(MAIL_REGEX, email)):
        return True
    else:
        print("Invalid Email.")
        return False

def insertMail():
    mail = input("Enter your email:")
    if checkMail(mail)==True:
        return mail
    else:
        print('\n')
        return insertMail()

In [2]:
def insertFirstname():
    return input('Enter your firstname:')

def insertLastname():
    return input('Enter your lastname:')

In [3]:
import hashlib
import getpass

def insertPassword(user, conn):
#     user is in the format (email,password)
    cursor = conn.cursor()
    oldPassword = getpass.getpass('Enter your old password:')
    hashedOldPassword=hashlib.sha256(bytes(oldPassword, encoding="ascii")).hexdigest()
    if (hashedOldPassword!=user[1]):
        return insertPassword(user, conn)
    else:
        newPassword = getpass.getpass('Enter your new password:')
        checkPassword = getpass.getpass("Verify password:")
        if newPassword == checkPassword :
            print("Password verified.")
            hashedNewPassword = hashlib.sha256(bytes(newPassword, encoding="ascii")).hexdigest()
            cursor.execute("UPDATE users SET password=%s WHERE email=%s",(hashedNewPassword,user[0]))
            conn.commit()
            print("Password updated successfully.")
        else:
            print("Password not verified.")
            print('\n')
            return insertPassword()

In [4]:
import psycopg2
#You need to create a database manually and then put the configuarion here
def databaseConfig():
    conn = psycopg2.connect(
        host="localhost",
        database="tp-securite",
        user="postgres",
        password="gh")

    cursor = conn.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS users (id SERIAL PRIMARY KEY, firstname VARCHAR(50), lastname varchar(50), email varchar(100) UNIQUE, password VARCHAR(255)); ")
    return conn


In [5]:
def signup(conn):
    firstname = insertFirstname()
    lastname = insertLastname()
    email = insertMail()
#     password = insertPassword() 
    password=generatePassword()
    hashedPassword= hashlib.sha256(bytes(password, encoding="ascii")).hexdigest()
    cursor = conn.cursor()
    cursor.execute("INSERT INTO users (firstname, lastname, email, password) VALUES (%s, %s, %s, %s)",
          (firstname, lastname, email, hashedPassword))
    conn.commit()
    print('User inserted successfully.\n')
    sendVerificationCode(email,firstname, password)

In [6]:
# Double factor verification
import smtplib, ssl
import sys,os
import math, random
sys.path.append(os.path.abspath('../CODE'))


port = 587  # For starttls
smtp_server = "smtp.office365.com"
sender_email = "ghassen.daoud@insat.u-carthage.tn"
# Passsword is removed because this file is published publicly in GitHub 
password="*****"

def generatePassword() :
    digits = "0123456789"
    password = ""
    for i in range(7) :
        password += digits[math.floor(random.random() * 10)]
    return password

def sendVerificationCode(receiver_email,receiver_name, otp):
    message = f"""\
Subject: Verification code

    Hello {receiver_name} This is your Password code: {otp} """
    context = ssl.create_default_context()
    with smtplib.SMTP(smtp_server, port) as server:
        server.ehlo()  # Can be omitted
        server.starttls(context=context)
        server.ehlo()  # Can be omitted
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, message)
        print('Please verify your email in order to get your password.')


In [7]:
import getpass

def login(conn):
    email=insertMail()
    password = getpass.getpass('Enter your password:')
    hashedPassword=hashlib.sha256(bytes(password, encoding="ascii")).hexdigest()
    cursor = conn.cursor()
    cursor.execute("SELECT email, password FROM users WHERE email=%s AND password=%s;", 
                   (email, hashedPassword))
    result=cursor.fetchone()
    if result:
        print ("User authentified.")
        return result
    else:
        print('Email or password incorrect.')
        return login(conn)

In [8]:
# CODAGE & DECODAGE

def encodeDecode():
    choice=input('Press 1 for encoding. \nPress 2 for decoding.\n')
    if choice != '1' and choice != '2':
        encodeDecode()
    if choice == '1':
        strOriginal = input('Please enter string to encode:\n')
        strEncoded = strOriginal.encode("ascii", "replace")
        print('Encoded bytes =', strEncoded)
        return strEncoded
    else:
        strOriginal = input('Please enter string to decode:\n')
        strOriginal = bytes(strOriginal, 'ascii')
        strDecoded = strOriginal.decode()
        print('Decoded String =', strOriginal)

        return strDecoded

In [9]:
#HACHAGE
import hashlib

def hashFunction():
    choice=input('Press 1 for hashing with MD5. \nPress 2 for hashing with SHA1.\nPress 3 for hashing with SHA256.\n')
    if choice != '1' and choice != '2' and choice != '3':
        return hashFunction()
    strOriginal = input('Enter password you want to hash:')
    if choice == '1':
        strHashed=hashlib.md5(bytes(strOriginal, encoding="ascii")).hexdigest()
    if choice == '2':
        strHashed=hashlib.sha1(bytes(strOriginal, encoding="ascii")).hexdigest()
    if choice == '3':
        strHashed=hashlib.sha256(bytes(strOriginal, encoding="ascii")).hexdigest()
    print('Your hashed password is:', strHashed)
    print()
    return strHashed

In [10]:
# passwords are in this format firstname.lastname@insat.ucar.tn
def crackInsatPassword():
    hashedMessage=input("Enter the password you want to hash:")
    FILE = open("5-6chars.txt","r")
    choice=input('Press 1 for cracking MD5. \nPress 2 for cracking SHA1.\nPress 3 for cracking SHA256.\n')
    if choice != '1' and choice != '2' and choice != '3':
        return crackInsatPassword(hashedMessage)
    break_flag=False
    if choice == '1': 
        for firstname in FILE:
            for lastname in FILE:
                password=firstname.strip()+'.'+lastname.strip()+'@insat.ucar.tn'
                if hashlib.md5(bytes(password, encoding="ascii")).hexdigest()==hashedMessage:
                    break_flag=True
                    break
            if break_flag:
                break
    if choice == '2': 
        for firstname in FILE:
            for lastname in FILE:
                password=firstname.strip()+'.'+lastname.strip()+'@insat.ucar.tn'
                if hashlib.sha1(bytes(password, encoding="ascii")).hexdigest()==hashedMessage:
                    break_flag=True
                    break
            if break_flag:
                break
    if choice == '3': 
        for firstname in FILE:
            for lastname in FILE:
                password=firstname.strip()+'.'+lastname.strip()+'@insat.ucar.tn'
                if hashlib.sha256(bytes(password, encoding="ascii")).hexdigest()==hashedMessage:
                    break_flag=True
                    break
            if break_flag:
                break
    print('Your message is: ', password)

In [11]:
# Symmetrical encryption
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding

def symmEncryption():
    global iv
    global cipher
    message=input('Enter the text you want to encrypt:')
    choice=input('Press 1 for AES encryption. \nPress 2 for 3DES encryption.\n')
    key = os.urandom(16)
    if choice != '1' and choice != '2':
        return symmEncryption(message)
    if choice == '1': 
        iv = os.urandom(16)
        cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    if choice == '2': 
        iv = os.urandom(8)
        cipher = Cipher(algorithms.TripleDES(key), modes.CBC(iv))
#     Padding
    padder = padding.ANSIX923(128).padder()
    padded_data = padder.update(bytes(message, encoding="ascii"))
    padded_data += padder.finalize()
#     Encryption
    encryptor = cipher.encryptor()
    encryptedMessage=encryptor.update(padded_data) + encryptor.finalize()
    print(encryptedMessage)
    return encryptedMessage,key,iv

def symmDecryption(message,key,iv):
    messageF=input("Enter the message you want to decrypt:").encode("ascii")
#     print(message)
    choice=input('Press 1 for AES decryption. \nPress 2 for 3DES decryption.\n')
    if choice != '1' and choice != '2':
        return symmDecryption(message)
    if choice == '1': 
        cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    if choice == '2': 
        cipher = Cipher(algorithms.TripleDES(key), modes.CBC(iv))
#     Decrypting
    decryptor = cipher.decryptor()
    decryptedMessage=decryptor.update(message) + decryptor.finalize()
#     Unpadding
    unpadder = padding.ANSIX923(128).unpadder()
    unpadded_message=unpadder.update(decryptedMessage) + unpadder.finalize()
    print('The decrypted message is: ',unpadded_message.decode())
    return unpadded_message


In [12]:
# Asymmetrical encryption
# first you need to run pip install elgamal
from elgamal.elgamal import Elgamal
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes


def genKeys():
    global prv_key, pub_key
    choice=input('Press 1 for RSA key generation. \nPress 2 for Elgamal key generation.\n')
    if choice != '1' and choice != '2':
        return genKeys()
    if choice=='1':
        prv_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
        
        prv_key_encoding = prv_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.PKCS8,
            encryption_algorithm=serialization.BestAvailableEncryption(b'mypassword')
        )
        pub_key = prv_key.public_key()
        pub_key_encoding = pub_key.public_bytes(
            encoding=serialization.Encoding.PEM,
           format=serialization.PublicFormat.SubjectPublicKeyInfo
        )
        print(prv_key_encoding)
        print(pub_key_encoding)
    if choice=='2':
        pub_key, prv_key = Elgamal.newkeys(128)
        print(prv_key)
        print()
        print(pub_key)
    return prv_key, pub_key


def asymEncryption(pub_key):
    global encrypted_message
    message=input("Enter the message you want to encrypt:")
    encoded_message=bytes(message, encoding="ascii")
    choice=input('Press 1 for RSA encryption. \nPress 2 for Elgamal encryption.\n')
    if choice != '1' and choice != '2':
        return asymEncryption(message)
    if choice=='1':
        encrypted_message = pub_key.encrypt(
            encoded_message,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
                )
            )
    if choice == '2': 
        encrypted_message = Elgamal.encrypt(encoded_message, pub_key)
    print(encrypted_message)
    return encrypted_message
    
    
def asymDecryption(message,prv_key):
    global decrypted_message
    messageF=input("Enter the message you want to decrypt:")
    choice=input('Press 1 for RSA encryption. \nPress 2 for Elgamal encryption.\n')
    if choice != '1' and choice != '2':
        return asymDecryption(message)
    if choice=='1':
        decrypted_message = prv_key.decrypt(
            message,
            padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
            )
        )
    if choice=='2':
        decrypted_message = Elgamal.decrypt(message, prv_key)
    print(decrypted_message)
    return decrypted_message


In [15]:
import psycopg2

def main():
    conn = databaseConfig()
    choice=input('Press 1 to sign up.\nPress 2 to login.\nPress 9 To exit.\n')
    if choice != '1' and choice != '2' and choice != '9':
        return main()
    if choice =='1':    
        signup(conn)
    if choice=='2':
        user=login(conn)
        while True:
            choice=input("""Press 1 to change password.
Press 2 to hash a password.
Press 3 to crack a hashed password.
Press 4 to encode/decode.
Press 5 for symmetrical encryption.
Press 6 for symmetrical decryption.
Press 7 to generate a pair of keys for assymetrical encryption.
Press 8 for assymetrical encryption. 
Press 9 for assymetrical decryption. 
Press 99 To exit.
                        """)                
            if choice=='1':
                insertPassword(user,conn)
            if choice=='2':
                hashedPassword=hashFunction()
            if choice=='3':
                crackInsatPassword()
            if choice=='4':
                encodeDecode()
            if choice=='5':
                symMessage,key,iv=symmEncryption()
            if choice=='6':
                symmDecryption(symMessage,key,iv)
            if choice=='7':
                prv_key,pub_key=genKeys()
            if choice=='8':
                asymMessage=asymEncryption(pub_key)
            if choice=='9':
                asymDecryption(asymMessage,prv_key)
            if choice=='99':
                return 
    if choice=='9':
        return
    
if __name__ == "__main__":
    main()

Press 1 to sign up.
Press 2 to login.
Press 9 To exit.
2
Enter your email:ghassendaoud99@gmail.com
Enter your password:········
User authentified.
Press 1 to change password.
Press 2 to hash a password.
Press 3 to crack a hashed password.
Press 4 to encode/decode.
Press 5 for symmetrical encryption.
Press 6 for symmetrical decryption.
Press 7 to generate a pair of keys for assymetrical encryption.
Press 8 for assymetrical encryption. 
Press 9 for assymetrical decryption. 
Press 99 To exit.
                        7
Press 1 for RSA key generation. 
Press 2 for Elgamal key generation.
2
PrivateKey(1929553074849427381518595688746622211947578605667451722387346952025076577419219036163402347066451430389266184892248131156287957427475678694682395216464214770509746261415517368369014803512789497234327762745766505166132253562457996705107161534911138756296060250690090162789648862645661461969764126928867506306938601088501739104650204098513492679295241334801739162149835517999366083976944330136815466

In [17]:
from tkinter import *
import getpass
import psycopg2
import hashlib
from PIL import Image, ImageTk
from tkinter import messagebox


root = Tk()

root.geometry("600x400")

 
ara = ImageTk.PhotoImage(Image.open("hack_bg.jpg"))
 
frame = Frame(root)
frame.rowconfigure(0, weight=1)
 
frame.columnconfigure(0, weight=1)
 
frame.pack(expand=1, fill=BOTH)
 
canvas = Canvas(frame)




 
canvas.grid(row=0, column=0, sticky=N+S+E+W)
 
canvas.create_image(0, 0, image=ara)
 
scroll_x = Scrollbar(frame, orient=HORIZONTAL, command=canvas.xview)
 
scroll_x.grid(row=1, column=0, sticky=E+W)
 
scroll_y = Scrollbar(frame, orient=VERTICAL, command=canvas.yview)
 
scroll_y.grid(row=0, column=1, sticky=N+S)
 
canvas["xscrollcommand"] = scroll_x.set
 
canvas["yscrollcommand"] = scroll_y.set
 
canvas["scrollregion"] = canvas.bbox(ALL)



# Input graphics
#Label 1
emailText = Label(canvas,text = 'Enter your email:',font=("Helvetica", 13), fg='red')
emailText.pack(pady=(80,6))
emailText.config(anchor="center")

email = Entry(canvas, width = 30)
email.pack(pady=(0,10))

passwordText = Label(canvas, text="Enter your password:",font=("Helvetica", 13),fg='red')
passwordText.pack(pady=(0,6))
passwordText.config(justify = CENTER)

password = Entry(canvas, width = 30)
password.config(show="*")
password.pack(pady=(0,14))



def login(email,password):
#     Database config
    conn = psycopg2.connect(
        host="localhost",
        database="tp-securite",
        user="postgres",
        password="gh")

    cursor = conn.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS users (id SERIAL PRIMARY KEY, firstname VARCHAR(50), lastname varchar(50), email varchar(100) UNIQUE, password VARCHAR(255)); ")
# Login
    user_email=email
    user_password = password
    hashedPassword=hashlib.sha256(bytes(user_password, encoding="ascii")).hexdigest()
    cursor = conn.cursor()
    cursor.execute("SELECT email, password FROM users WHERE email=%s AND password=%s;", 
                   (user_email, hashedPassword))
    result=cursor.fetchone()
    if result:
        print ("User authentified.")
        messagebox.showinfo( "Success", "You're logged in")
        return result
    else:
        print('Email or password incorrect.')
        messagebox.showerror("Error", "Email or password are incorrect")
        return login(conn)


    

submit=Button(canvas, text ="Login", command=lambda: login(email.get(),password.get()),font=("Helvetica", 12))
submit.pack()
root.mainloop()



    
 
